In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten 
from keras.layers import MaxPooling2D, Dropout
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
def getTrainingSetSize(df, noOfcells):
    trainingDataPointSize = int(df.shape[0]*0.8/noOfcells)
    training_setsize = trainingDataPointSize*noOfcells
    return training_setsize

In [3]:
def getProcessedDataSet(df):
    input_X = []
    input_Y = []
    dataPoint_X = []
    dataPoint_Y = []
    scaler = StandardScaler()


    for index, rows in df.iterrows():
        if index == 0:
            X_row_list = [rows.InputVehicleId, rows.InputVehicleType, rows.InputSignalGroup, rows.InputLaneId, rows.InputApproachId, rows.InputPhaseStatus, rows.InputSpeed, rows.InputHeading, rows.InputDistanceToStopBar]
#             X_row_list = [rows.InputVehicleId, rows.InputVehicleType, rows.InputSignalGroup, rows.InputLaneId, rows.InputApproachId, rows.InputLocationOnMap,
#                     rows.InputPhaseStatus, rows.InputSpeed, rows.InputHeading, rows.InputDistanceToStopBar, rows.InputStoppedDelay, rows.InputCellStatus]
            
#             Y_row_list = [rows.OutputCellStatus]
            
            dataPoint_X.append(X_row_list)
            dataPoint_Y.append(rows.InputCellStatus)
        
        elif index % noOfcells != 0:
            X_row_list = [rows.InputVehicleId, rows.InputVehicleType, rows.InputSignalGroup, rows.InputLaneId, rows.InputApproachId, rows.InputPhaseStatus, rows.InputSpeed, rows.InputHeading, rows.InputDistanceToStopBar]
#             X_row_list = [rows.InputVehicleId, rows.InputVehicleType, rows.InputSignalGroup, rows.InputLaneId, rows.InputApproachId, rows.InputLocationOnMap,
#                     rows.InputPhaseStatus, rows.InputSpeed, rows.InputHeading, rows.InputDistanceToStopBar, rows.InputStoppedDelay, rows.InputCellStatus]
            
#             Y_row_list = [rows.OutputVehicleId, rows.OutputVehicleType, rows.OutputSignalGroup, rows.OutputLaneId, rows.OutputApproachId, rows.OutputLocationOnMap,
#                     rows.OutputPhaseStatus, rows.OutputSpeed, rows.OutputHeading, rows.OutputDistanceToStopBar, rows.OutputStoppedDelay, rows.OutputCellStatus]
            
            dataPoint_X.append(X_row_list)
            dataPoint_Y.append(rows.InputCellStatus)

        elif index % noOfcells == 0 and len(dataPoint_X) > 0:
            scaler = scaler.fit(dataPoint_X)
            dataPoint_X = scaler.transform(dataPoint_X)
            input_X.append(dataPoint_X)
            input_Y.append(dataPoint_Y)
            
            dataPoint_X = []
            dataPoint_Y = []
            X_row_list = [rows.InputVehicleId, rows.InputVehicleType, rows.InputSignalGroup, rows.InputLaneId, rows.InputApproachId, rows.InputPhaseStatus, rows.InputSpeed, rows.InputHeading, rows.InputDistanceToStopBar]
#             X_row_list = [rows.InputVehicleId, rows.InputVehicleType, rows.InputSignalGroup, rows.InputLaneId, rows.InputApproachId, rows.InputLocationOnMap,
#                     rows.InputPhaseStatus, rows.InputSpeed, rows.InputHeading, rows.InputDistanceToStopBar, rows.InputStoppedDelay, rows.InputCellStatus]
            
#             Y_row_list = [rows.OutputVehicleId, rows.OutputVehicleType, rows.OutputSignalGroup, rows.OutputLaneId, rows.OutputApproachId, rows.OutputLocationOnMap,
#                     rows.OutputPhaseStatus, rows.OutputSpeed, rows.OutputHeading, rows.OutputDistanceToStopBar, rows.OutputStoppedDelay, rows.OutputCellStatus]
            
            dataPoint_X.append(X_row_list)
            dataPoint_Y.append(rows.InputCellStatus)
        
        if index == len(df) - 1:
            scaler = scaler.fit(dataPoint_X)
            dataPoint_X = scaler.transform(dataPoint_X)
            input_X.append(dataPoint_X)
            input_Y.append(dataPoint_Y)  


    input_X = np.array(input_X)
    input_Y = np.array(input_Y)
    
    
    return input_X, input_Y

In [4]:
df = pd.read_csv('processed-data.csv')
#df = pd.read_csv('vehicle-status-prediction-data.csv')
# df = pd.read_csv('test-data.csv')
noOfcells = df['NoOfCells'][0]
print(df.shape[0])

489528


In [5]:
training_setsize = getTrainingSetSize(df, noOfcells)

df_forTraining = df.iloc[:training_setsize, 1:]

df_forTesting = df.iloc[training_setsize:, 1:]
df_forTesting.reset_index(inplace=True)

In [6]:
df_forTesting.head()

,index,NoOfCells,InputVehicleId,InputVehicleType,InputSignalGroup,InputLaneId,InputApproachId,InputPhaseStatus,InputSpeed,InputHeading,InputDistanceToStopBar,InputCellStatus,OutputCellStatus
0,391560,156,0,0,5,15,5,4,0.0,0.0,0,0,0
1,391561,156,0,0,5,15,5,4,0.0,0.0,0,0,0
2,391562,156,0,0,5,15,5,4,0.0,0.0,0,0,0
3,391563,156,0,0,5,15,5,4,0.0,0.0,0,0,0
4,391564,156,0,0,5,15,5,4,0.0,0.0,0,0,0


In [7]:
trainX, trainY = getProcessedDataSet(df_forTraining)

print(trainX.shape)
print(trainY.shape)

(2510, 156, 9)
(2510, 156)


In [8]:
trainX = trainX.reshape(trainX.shape[0], trainX.shape[1], trainX.shape[2], 1)
print(trainX.shape)
print(trainY.shape)

(2510, 156, 9, 1)
(2510, 156)


In [9]:
testX, testY = getProcessedDataSet(df_forTesting)
testX = testX.reshape(testX.shape[0], testX.shape[1], testX.shape[2], 1)

In [10]:
print(testX.shape)

(628, 156, 9, 1)


In [11]:
testX

array([[[[ 0.        ],
         [ 0.        ],
         [ 3.06594194],
         ...,
         [ 0.        ],
         [ 0.        ],
         [ 0.        ]],

        [[ 0.        ],
         [ 0.        ],
         [ 3.06594194],
         ...,
         [ 0.        ],
         [ 0.        ],
         [ 0.        ]],

        [[ 0.        ],
         [ 0.        ],
         [ 3.06594194],
         ...,
         [ 0.        ],
         [ 0.        ],
         [ 0.        ]],

        ...,

        [[ 0.        ],
         [ 0.        ],
         [-0.32616404],
         ...,
         [ 0.        ],
         [ 0.        ],
         [ 0.        ]],

        [[ 0.        ],
         [ 0.        ],
         [-0.32616404],
         ...,
         [ 0.        ],
         [ 0.        ],
         [ 0.        ]],

        [[ 0.        ],
         [ 0.        ],
         [-0.32616404],
         ...,
         [ 0.        ],
         [ 0.        ],
         [ 0.        ]]],


       [[[ 0.        ],


In [12]:
input_shape = (trainX.shape[1], trainX.shape[2], 1)


In [13]:
model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
# model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(156, activation='softmax'))#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
#train the model
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=10)

Train on 2510 samples, validate on 628 samples
Epoch 1/10
2510/2510 [==============================] - 3s 1ms/step - loss: 3.5523 - accuracy: 0.3884 - val_loss: 5.0418 - val_accuracy: 0.3105
Epoch 2/10
2510/2510 [==============================] - 3s 1ms/step - loss: 74.1827 - accuracy: 0.2335 - val_loss: 43.6407 - val_accuracy: 0.3105
Epoch 3/10
2510/2510 [==============================] - 3s 1ms/step - loss: 351.2439 - accuracy: 0.2036 - val_loss: 195.8557 - val_accuracy: 0.3105
Epoch 4/10
2510/2510 [==============================] - 3s 1ms/step - loss: 880.3437 - accuracy: 0.1849 - val_loss: 522.3672 - val_accuracy: 0.7325
Epoch 5/10
2510/2510 [==============================] - 3s 1ms/step - loss: 1674.3105 - accuracy: 0.1781 - val_loss: 950.7197 - val_accuracy: 0.1481
Epoch 6/10
2510/2510 [==============================] - 3s 1ms/step - loss: 2637.1519 - accuracy: 0.1610 - val_loss: 1459.9853 - val_accuracy: 0.1847
Epoch 7/10
2510/2510 [==============================] - 3s 1ms/step 

In [15]:
# evaluate the model
score = model.evaluate(testX, testY, verbose=1)

628/628 [==============================] - 0s 227us/step


In [16]:

# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])
#predict first 4 images in the test set
model.predict(testX[:4])
model.predict_classes(testX[:4])
#actual results for first 4 data points in test set
testY[:4]


Test loss: 4077.2923964968154
Test accuracy: 0.1624203771352768


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       